# Analise de Teste A/B - recommender_system_test

## Introducao

Este projeto tem como objetivo analisar os resultados de um teste A/B realizado por uma loja online internacional para avaliar o impacto de um sistema de recomendacao aprimorado. O experimento compara um grupo de controle (A) com um grupo de teste (B), medindo diferencas nas taxas de conversao ao longo do funil de usuarios — visualização de pagina do produto, adicao ao carrinho e compra — dentro de até 14 dias apos o cadastro. Alem da avaliacao estatística dos resultados, o projeto também verifica se o experimento foi conduzido corretamente em termos de publico, período e qualidade dos dados.

## Importacao das Blibliotecas

In [1]:
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest

In [2]:

marketing_events = pd.read_csv('/datasets/ab_project_marketing_events_us.csv')
new_users = pd.read_csv('/datasets/final_ab_new_users_upd_us.csv')
events_upd = pd.read_csv('/datasets/final_ab_events_upd_us.csv')
participants_upd = pd.read_csv('/datasets/final_ab_participants_upd_us.csv')


## Exploracao Inicial dos Dados

### Carregamento dos Dados

In [3]:
#Informacoes Gerais do Dataset

print('\n --- Primeiras linhas do Dataset ---')
print(marketing_events.head())
print('\n --- Informacoes Gerais do Dataset ---')
print(marketing_events.info())



 --- Primeiras linhas do Dataset ---
                           name                   regions    start_dt  \
0      Christmas&New Year Promo             EU, N.America  2020-12-25   
1  St. Valentine's Day Giveaway  EU, CIS, APAC, N.America  2020-02-14   
2        St. Patric's Day Promo             EU, N.America  2020-03-17   
3                  Easter Promo  EU, CIS, APAC, N.America  2020-04-12   
4             4th of July Promo                 N.America  2020-07-04   

    finish_dt  
0  2021-01-03  
1  2020-02-16  
2  2020-03-19  
3  2020-04-19  
4  2020-07-11  

 --- Informacoes Gerais do Dataset ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 576.0+ bytes
None


In [4]:
#Informacoes Gerais do Dataset

print('\n --- Primeiras linhas do Dataset ---')
print(new_users.head())
print('\n --- Informacoes Gerais do Dataset ---')
print(new_users.info())


 --- Primeiras linhas do Dataset ---
            user_id  first_date     region   device
0  D72A72121175D8BE  2020-12-07         EU       PC
1  F1C668619DFE6E65  2020-12-07  N.America  Android
2  2E1BF1D4C37EA01F  2020-12-07         EU       PC
3  50734A22C0C63768  2020-12-07         EU   iPhone
4  E1BDDCE0DAFA2679  2020-12-07  N.America   iPhone

 --- Informacoes Gerais do Dataset ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58703 entries, 0 to 58702
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     58703 non-null  object
 1   first_date  58703 non-null  object
 2   region      58703 non-null  object
 3   device      58703 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB
None


In [5]:
#Informacoes Gerais do Dataset

print('\n --- Primeiras linhas do Dataset ---')
print(events_upd.head())
print('\n --- Informacoes Gerais do Dataset ---')
print(events_upd.info())


 --- Primeiras linhas do Dataset ---
            user_id             event_dt event_name  details
0  E1BDDCE0DAFA2679  2020-12-07 20:22:03   purchase    99.99
1  7B6452F081F49504  2020-12-07 09:22:53   purchase     9.99
2  9CD9F34546DF254C  2020-12-07 12:59:29   purchase     4.99
3  96F27A054B191457  2020-12-07 04:02:40   purchase     4.99
4  1FD7660FDF94CA1F  2020-12-07 10:15:09   purchase     4.99

 --- Informacoes Gerais do Dataset ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423761 entries, 0 to 423760
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     423761 non-null  object 
 1   event_dt    423761 non-null  object 
 2   event_name  423761 non-null  object 
 3   details     60314 non-null   float64
dtypes: float64(1), object(3)
memory usage: 12.9+ MB
None


In [6]:
#Informacoes Gerais do Dataset

print('\n --- Primeiras linhas do Dataset ---')
print(participants_upd.head())
print('\n --- Informacoes Gerais do Dataset ---')
print(participants_upd.info())


 --- Primeiras linhas do Dataset ---
            user_id group                  ab_test
0  D1ABA3E2887B6A73     A  recommender_system_test
1  A7A3664BD6242119     A  recommender_system_test
2  DABC14FDDFADD29E     A  recommender_system_test
3  04988C5DF189632E     A  recommender_system_test
4  4FF2998A348C484F     A  recommender_system_test

 --- Informacoes Gerais do Dataset ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  14525 non-null  object
 1   group    14525 non-null  object
 2   ab_test  14525 non-null  object
dtypes: object(3)
memory usage: 340.6+ KB
None


### Conclusao da Exploracao Inicial dos Dados
Foram analisados quatro conjuntos de dados relacionados ao experimento A/B. O dataset de eventos de marketing contem 14 campanhas ao longo de 2020, sem valores ausentes, mas com colunas de data que exigem conversao para o tipo datetime. A base de novos usuários possui 58.703 registros, todos completos, incluindo informacoes de data de cadastro, regiao e dispositivo, tambem necessitando conversao de datas. O dataset de eventos apresenta 423.761 registros de interação dos usuários; a coluna details possui valores apenas para eventos de compra, o que é esperado e não caracteriza problema de qualidade. Por fim, a tabela de participantes do teste contém 14.525 usuários, todos corretamente associados ao teste e aos grupos A ou B. De forma geral, os dados estão completos, consistentes e adequados para as próximas etapas da análise, exigindo apenas ajustes de tipos de dados.

## Preparacao dos Dados para Analise

### Conversao dos Tipos de Dados

In [7]:
#Convertendo para datetime

#Marketing
marketing_events['start_dt'] = pd.to_datetime(marketing_events['start_dt'])
marketing_events['finish_dt'] = pd.to_datetime(marketing_events['finish_dt'])

#Novos usuarios
new_users['first_date'] = pd.to_datetime(new_users['first_date'])

#eventos
events_upd['event_dt'] = pd.to_datetime(events_upd['event_dt'])


print(marketing_events.dtypes)
print(new_users.dtypes)
print(events_upd.dtypes)

name                 object
regions              object
start_dt     datetime64[ns]
finish_dt    datetime64[ns]
dtype: object
user_id               object
first_date    datetime64[ns]
region                object
device                object
dtype: object
user_id               object
event_dt      datetime64[ns]
event_name            object
details              float64
dtype: object


In [8]:
#Filtrando apenas o teste relevante

participants_upd = participants_upd[participants_upd['ab_test'] == 'recommender_system_test']

print(participants_upd['ab_test'].value_counts())
print(participants_upd['group'].value_counts())

recommender_system_test    3675
Name: ab_test, dtype: int64
A    2747
B     928
Name: group, dtype: int64


In [9]:
#Garantindo usuarios de teste existem 'new_users'
participants_users = participants_upd.merge(new_users, on='user_id', how='inner')

print(participants_users.shape)
participants_users.head()

(3675, 6)


,user_id,group,ab_test,first_date,region,device
0,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC
1,A7A3664BD6242119,A,recommender_system_test,2020-12-20,EU,iPhone
2,DABC14FDDFADD29E,A,recommender_system_test,2020-12-08,EU,Mac
3,04988C5DF189632E,A,recommender_system_test,2020-12-14,EU,iPhone
4,4FF2998A348C484F,A,recommender_system_test,2020-12-20,EU,Mac


In [10]:
#Filtrando apenas usuarios EU
participants_users = participants_users[participants_users['region'] =='EU']

print(participants_users['region'].value_counts())
print(participants_users['group'].value_counts())

EU    3481
Name: region, dtype: int64
A    2604
B     877
Name: group, dtype: int64


In [11]:
#Filtrando eventos desses usuarios

events_test = events_upd.merge(participants_users[['user_id', 'group', 'first_date']], on='user_id', how='inner')

print(events_test.shape)
events_test.head()

(22674, 6)


,user_id,event_dt,event_name,details,group,first_date
0,831887FE7F2D6CBA,2020-12-07 06:50:29,purchase,4.99,A,2020-12-07
1,831887FE7F2D6CBA,2020-12-09 02:19:17,purchase,99.99,A,2020-12-07
2,831887FE7F2D6CBA,2020-12-07 06:50:30,product_cart,NaN,A,2020-12-07
3,831887FE7F2D6CBA,2020-12-08 10:52:27,product_cart,NaN,A,2020-12-07
4,831887FE7F2D6CBA,2020-12-09 02:19:17,product_cart,NaN,A,2020-12-07


In [12]:
#criando janelas de 14 dias

events_test['days_since_signup'] = (events_test['event_dt'] - events_test['first_date']).dt.days

print(events_test['days_since_signup'].describe())

count    22674.000000
mean         3.150613
std          4.050777
min          0.000000
25%          0.000000
50%          2.000000
75%          5.000000
max         23.000000
Name: days_since_signup, dtype: float64


In [13]:
events_test = events_test[
    (events_test['days_since_signup'] >= 0) &
    (events_test['days_since_signup'] <= 14)]

print(events_test['days_since_signup'].min(),
      events_test['days_since_signup'].max())

0 14


A preparacao dos dados foi concluída com sucesso, assegurando a correta tipagem das variaveis, a aplicação dos filtros do experimento e a definicao da janela temporal de analise. O dataset final está consistente e pronto para a analise exploratória do teste A/B.

# Analise Exploratoria de Dados

In [14]:
#Checagens basicas do dataset final

print("participants_users:", participants_users.shape)
print(participants_users.head())

print("\nevents_test:", events_test.shape)
print(events_test.head())

print("\nPeríodo de eventos (min/max):")
print(events_test['event_dt'].min(), "→", events_test['event_dt'].max())

participants_users: (3481, 6)
            user_id group                  ab_test first_date region  device
0  D1ABA3E2887B6A73     A  recommender_system_test 2020-12-07     EU      PC
1  A7A3664BD6242119     A  recommender_system_test 2020-12-20     EU  iPhone
2  DABC14FDDFADD29E     A  recommender_system_test 2020-12-08     EU     Mac
3  04988C5DF189632E     A  recommender_system_test 2020-12-14     EU  iPhone
4  4FF2998A348C484F     A  recommender_system_test 2020-12-20     EU     Mac

events_test: (22157, 7)
            user_id            event_dt    event_name  details group  \
0  831887FE7F2D6CBA 2020-12-07 06:50:29      purchase     4.99     A   
1  831887FE7F2D6CBA 2020-12-09 02:19:17      purchase    99.99     A   
2  831887FE7F2D6CBA 2020-12-07 06:50:30  product_cart      NaN     A   
3  831887FE7F2D6CBA 2020-12-08 10:52:27  product_cart      NaN     A   
4  831887FE7F2D6CBA 2020-12-09 02:19:17  product_cart      NaN     A   

  first_date  days_since_signup  
0 2020-12-07    

In [15]:
#presenca dos usuarios e equilibrio entre grupos

users_by_group = participants_users.groupby('group')['user_id'].nunique()
print("Usuarios únicos por grupo:")
print(users_by_group)

print("\nProporcao por grupo:")
print((users_by_group / users_by_group.sum()).round(4))

Usuarios únicos por grupo:
group
A    2604
B     877
Name: user_id, dtype: int64

Proporcao por grupo:
group
A    0.7481
B    0.2519
Name: user_id, dtype: float64


In [16]:
#garantir que nao existe o mesmo 'user_id' em A e B

groups_per_user = participants_users.groupby('user_id')['group'].nunique()
bad_users = groups_per_user[groups_per_user > 1]

print("Usuarios em mais de um grupo:", bad_users.shape[0])
print(bad_users.head())

Usuarios em mais de um grupo: 0
Series([], Name: group, dtype: int64)


In [17]:
#Quais tipos de eventos existem e em que volumes

print("Tipos de evento (contagem):")
print(events_test['event_name'].value_counts().head(20))

print("\nTipos de evento (usuários únicos por evento):")
print(events_test.groupby('event_name')['user_id'].nunique().sort_values(ascending=False).head(20))

Tipos de evento (contagem):
login           10013
product_page     6191
purchase         3018
product_cart     2935
Name: event_name, dtype: int64

Tipos de evento (usuários únicos por evento):
event_name
login           3480
product_page    2178
purchase        1082
product_cart    1026
Name: user_id, dtype: int64


In [18]:
# Eventos por usuario e comparacao entre grupos
events_per_user = (
    events_test
    .groupby(['group', 'user_id'])
    .size()
    .reset_index(name='events_count')
)

print("Resumo eventos por usuario (por grupo):")
print(events_per_user.groupby('group')['events_count'].describe())

Resumo eventos por usuario (por grupo):
        count      mean       std  min  25%  50%  75%   max
group                                                      
A      2604.0  6.673195  3.701340  1.0  4.0  6.0  9.0  24.0
B       877.0  5.450399  3.297829  1.0  3.0  4.0  7.0  24.0


In [19]:
#Distribuicao de eventos ao longo do dia

events_test['event_date'] = events_test['event_dt'].dt.date

events_by_day = (
    events_test
    .groupby(['event_date', 'group'])
    .size()
    .reset_index(name='events_count')
)

print(events_by_day.head())
print(events_by_day.tail())

print("\nResumo diario de eventos por grupo:")
print(events_by_day.groupby('group')['events_count'].describe())

   event_date group  events_count
0  2020-12-07     A           318
1  2020-12-07     B           356
2  2020-12-08     A           313
3  2020-12-08     B           238
4  2020-12-09     A           371
    event_date group  events_count
39  2020-12-27     B            66
40  2020-12-28     A           452
41  2020-12-28     B            69
42  2020-12-29     A           291
43  2020-12-29     B            41

Resumo diario de eventos por grupo:
       count        mean         std    min     25%    50%      75%     max
group                                                                      
A       22.0  789.863636  475.975519  291.0  348.50  674.5  1157.50  1903.0
B       22.0  217.272727  104.974002   41.0  161.75  230.0   286.25   401.0


In [20]:
#Funil: usuarios unico por etapa

funnel_events = ['product_page', 'product_cart', 'purchase']

users_per_event = (
    events_test[events_test['event_name'].isin(funnel_events)]
    .groupby('event_name')['user_id']
    .nunique()
    .reindex(funnel_events)
)

print("Usuarios unicos por etapa (geral):")
print(users_per_event)

Usuarios unicos por etapa (geral):
event_name
product_page    2178
product_cart    1026
purchase        1082
Name: user_id, dtype: int64


In [21]:
#Funil por grupo

users_per_event_group = (
    events_test[events_test['event_name'].isin(funnel_events)]
    .groupby(['group', 'event_name'])['user_id']
    .nunique()
    .reset_index()
)

print("Usuários únicos por etapa e grupo:")
print(users_per_event_group)

Usuários únicos por etapa e grupo:
  group    event_name  user_id
0     A  product_cart      782
1     A  product_page     1685
2     A      purchase      833
3     B  product_cart      244
4     B  product_page      493
5     B      purchase      249


A analise exploratoria identificou algumas particularidades relevantes que devem ser consideradas antes da avaliação estatística do teste A/B. Primeiramente, os grupos do experimento apresentam um desbalanceamento significativo, com aproximadamente 75% dos usuários no grupo A e 25% no grupo B, o que pode reduzir o poder estatístico das comparações. Alem disso, observou-se que o funil de conversao nao é estritamente linear, uma vez que eventos de compra ocorrem mesmo na ausência do evento de adição ao carrinho, indicando a existência de fluxos de compra direta. Por fim, constatou-se que os usuários geram múltiplos eventos dentro da janela de análise, o que reforca a necessidade de utilizar metricas baseadas em usuários únicos, e não em volume total de eventos, para evitar vieses na interpretação dos resultados.

# Avaliacao do Teste A/B

## Definicao das Metricas de Conversao

Com base nos resultados da analise exploratoria, definiu-se que as métricas de conversão seriam avaliadas de forma independente para cada evento do funil (product_page, product_cart e purchase). Essa decisao foi tomada devido à nao linearidade observada no funil de conversao, uma vez que eventos de compra podem ocorrer mesmo na ausencia do evento de adicao ao carrinho. As taxas de conversao foram calculadas considerando usuarios únicos em cada grupo do experimento, garantindo uma comparacao adequada entre os grupos A e B.

In [22]:
# Tamanho da Amostra por Grupo
users_by_group = participants_users.groupby('group')['user_id'].nunique()
print(users_by_group)

group
A    2604
B     877
Name: user_id, dtype: int64


In [23]:
#Tabela de Conversao por exemplo

funnel_events = ['product_page', 'product_cart', 'purchase']

conversion_table = []

for event in funnel_events:
    users_event = (
        events_test[events_test['event_name'] == event]
        .groupby('group')['user_id']
        .nunique()
    )
    
    for group in ['A', 'B']:
        conversion_table.append({
            'event': event,
            'group': group,
            'users_converted': users_event.get(group, 0),
            'users_total': users_by_group[group]
        })

conversion_df = pd.DataFrame(conversion_table)
conversion_df['conversion_rate'] = (
    conversion_df['users_converted'] / conversion_df['users_total']
)

print(conversion_df)

          event group  users_converted  users_total  conversion_rate
0  product_page     A             1685         2604         0.647081
1  product_page     B              493          877         0.562144
2  product_cart     A              782         2604         0.300307
3  product_cart     B              244          877         0.278221
4      purchase     A              833         2604         0.319892
5      purchase     B              249          877         0.283922


In [24]:
# Implementacao do z-test de proporcoes

test_results = []

for event in funnel_events:
    a = conversion_df.query("event == @event and group == 'A'")
    b = conversion_df.query("event == @event and group == 'B'")
    
    count = [int(b['users_converted']), int(a['users_converted'])]
    nobs = [int(b['users_total']), int(a['users_total'])]
    
    z_stat, p_value = proportions_ztest(
        count, nobs, alternative='larger'
    )
    
    test_results.append({
        'event': event,
        'conversion_A': float(a['conversion_rate']),
        'conversion_B': float(b['conversion_rate']),
        'uplift_%': (float(b['conversion_rate']) / float(a['conversion_rate']) - 1) * 100,
        'z_stat': z_stat,
        'p_value': p_value
    })

results_df = pd.DataFrame(test_results)
print(results_df)

          event  conversion_A  conversion_B   uplift_%    z_stat   p_value
0  product_page      0.647081      0.562144 -13.126284 -4.495436  0.999997
1  product_cart      0.300307      0.278221  -7.354472 -1.240767  0.892654
2      purchase      0.319892      0.283922 -11.244407 -1.990600  0.976738


#### Interpretacao dos Resultados do Teste A/B


A avaliacao estatística do teste A/B foi realizada por meio de testes z para comparacao de proporcoes entre os grupos A (controle) e B (novo sistema de recomendacao), considerando um nivel de significância de 5%.

Para o evento product_page, a taxa de conversao do grupo B (56,21%) foi inferior à do grupo A (64,71%), resultando em um uplift negativo de aproximadamente –13,1%. O teste estatístico indicou que essa diferença não é estatisticamente significativa (p-value = 0,999997), não atendendo ao critério de sucesso definido para o experimento.


No evento product_cart, observou-se novamente uma conversao menor no grupo B (27,82%) em comparação ao grupo A (30,03%), com um uplift de –7,35%. Essa dife
rença também não foi estatisticamente significativa (p-value = 0,892654), alem de nao atingir o aumento mínimo de 10% esperado.

Em relacao ao evento purchase, o grupo B apresentou uma taxa de conversão de 28,39%, inferior à do grupo A (31,99%), correspondendo a um uplift negativo de –11,24%. Embora o valor do estatístico z indique uma diferença mais próxima do limiar crítico, o resultado não foi estatisticamente significativo (p-value = 0,976738) e não satisfaz o critério de negocio estabelecido.

De forma geral, os resultados indicam que o novo sistema de recomendacao nao apresentou melhoria nas taxas de conversao em nenhuma das etapas avaliadas do funil. Alem de nao atingir o aumento mínimo de 10% esperado, o grupo B apresentou desempenho inferior ao grupo A em todas as métricas analisadas, não havendo evidências estatísticas que justifiquem a adocao da nova versão com base nos dados deste experimento.

#Conclusao Final

Com base nos resultados obtidos, recomenda-se nao implementar o novo sistema de recomendação na forma testada, uma vez que ele nao demonstrou ganhos estatisticamente significativos nem melhorias praticas nas conversões dos usuarios.